In [1]:
!git clone https://github.com/ultralytics/flickr_scraper
%cd flickr_scraper
!pip install -U -r requirements.txt

Cloning into 'flickr_scraper'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 186 (delta 0), reused 1 (delta 0), pack-reused 182
Receiving objects: 100% (186/186), 46.52 KiB | 15.51 MiB/s, done.
Resolving deltas: 100% (113/113), done.
/content/flickr_scraper
     |████████████████████████████████| 15.7 MB 74 kB/s 
     |████████████████████████████████| 3.0 MB 55.8 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Su

In [1]:
%cd /content/flickr_scraper

/content/flickr_scraper


In [ ]:
!python3 flickr_scraper.py --search 'flood' --n 200 --download


In [ ]:
!python3 flickr_scraper.py --search 'fire' --n 200 --download


In [ ]:
!python3 flickr_scraper.py --search 'cars crash' --n 200 --download


In [ ]:
!python3 flickr_scraper.py --search 'dead person' --n 200 --download


In [8]:
!pip install -q tflite-model-maker
import os
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import matplotlib.pyplot as plt

     |████████████████████████████████| 591 kB 29.8 MB/s 
     |████████████████████████████████| 1.2 MB 61.2 MB/s 
     |████████████████████████████████| 6.3 MB 72.9 MB/s 
     |████████████████████████████████| 636 kB 65.6 MB/s 
     |████████████████████████████████| 120 kB 34.9 MB/s 
     |████████████████████████████████| 103 kB 80.4 MB/s 
     |████████████████████████████████| 840 kB 47.3 MB/s 
     |████████████████████████████████| 64 kB 3.8 MB/s 
     |████████████████████████████████| 679 kB 49.6 MB/s 
     |████████████████████████████████| 87 kB 7.9 MB/s 
     |████████████████████████████████| 211 kB 55.3 MB/s 
     |████████████████████████████████| 1.1 MB 69.9 MB/s 
     |████████████████████████████████| 352 kB 73.4 MB/s 
     |████████████████████████████████| 37.1 MB 48 kB/s 
     |████████████████████████████████| 99 kB 9.8 MB/s 
     |████████████████████████████████| 14.8 MB 146 kB/s 
     |████████████████████████████████| 199 kB 66.4 MB/s 
ERROR: pip's dependen

In [9]:
image_path='/content/flickr_scraper/images'
data = DataLoader.from_folder(image_path)
#train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 800, num_label: 4, labels: cars_crash, dead_person, fire, flood.


In [ ]:
#model = image_classifier.create(train_data)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

model = image_classifier.create(train_data, validation_data=validation_data, epochs=10)


INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 3,418,148
Trainable params: 5,124
Non-trainable params: 3,413,024
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/10
 4/20 [=====>........................] - ETA: 44s - loss: 1.5145 - accuracy: 0.2891

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(30, 30))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(36)):
  ax = plt.subplot(6, 6, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

In [ ]:
model.export(export_dir='/content/model16', export_format=ExportFormat.LABEL)
config = QuantizationConfig.for_float16()
model.export(export_dir='/content/model16', tflite_filename='model_events.tflite', quantization_config=config)